---
layout: post
title: Sprint5 Blog
description:  Leaderboard
type: issues 
comments: true
---

Group
The purpose of our group’s program is to create a multiplayer drawing and guessing game inspired by Scribble.io. This program allows players to take turns drawing while others guess the word or phrase being illustrated. The goal is to provide an entertaining and interactive way for people to engage in creative gameplay, fostering collaboration, quick thinking, and creativity.

Individual
My individual feature is primarily focused on making a leaderboard to showcase your scores. This feature allows people to see their highscores, low scores, and challenge themselves with default scores to earn a better score. The purpose of the feature is to also allow for a more gamelike experience for players.



## Fullstack


# Leaderboard API Technical Analysis

## 1. List Requests, Use of Lists, Dictionaries, and Database




## List Requests  
The `get` method retrieves all leaderboard entries and returns them as a JSON response.  

## Use of Lists and Dictionaries  
The `entries` variable is a list of `LeaderboardEntry` objects, and each entry is converted to a dictionary format for the JSON response.  

## Formatting Response Data  

- When the API retrieves data from the database, it formats the results as a list of dictionaries, where:

- Each row in the database represents a list entry.
- Each column in the database is stored as key-value pairs in a dictionary.

## Database Queries  
SQLAlchemy is used to query the database, returning Python lists representing rows of data.  

## Methods in Class  
The `_LeaderboardCRUD` class defines methods to create, read, update, and delete entries, working with the columns of the database.  


In [ ]:
# Import necessary modules from Flask and other dependencies
from flask import Blueprint, request, jsonify, current_app, Response, g
from flask_restful import Api, Resource
from datetime import datetime
from __init__ import app
from api.jwt_authorize import token_required
from model.leaderboard import LeaderboardEntry

# Create Blueprint for API routing with '/api' prefix
leaderboard_api = Blueprint('leaderboard_api', __name__, url_prefix='/api')
api = Api(leaderboard_api)

class LeaderboardAPI:
    class _CRUD(Resource):
        @token_required()  # Decorator to ensure user is authenticated
        def post(self):
            """Create a new leaderboard entry with authentication"""
            # Get current user from global context
            current_user = g.current_user
            data = request.get_json()

            # Validate required fields in request
            if not data or "drawing_name" not in data or "score" not in data:
                return jsonify({
                    "message": "Missing required fields",
                    "error": "Bad Request"
                }), 400

            try:
                # Create new LeaderboardEntry with user information
                entry = LeaderboardEntry(
                    profile_name=current_user.name,
                    drawing_name=data['drawing_name'],
                    score=int(data['score']),
                    created_by=current_user.id
                )
                entry.create()
                return jsonify(entry.read()), 201

            except Exception as e:
                return jsonify({
                    "message": "Failed to create entry",
                    "error": str(e)
                }), 500

        def get(self):
            """Retrieve all leaderboard entries sorted by score"""
            try:
                # Query database for all entries, ordered by score descending
                entries = LeaderboardEntry.query.order_by(
                    LeaderboardEntry.score.desc()
                ).all()
                return jsonify([entry.read() for entry in entries])

            except Exception as e:
                return jsonify([])  # Return empty list if query fails

        @token_required()
        def delete(self):
            """Delete a leaderboard entry (Admin only)"""
            current_user = g.current_user
            
            # Verify user has admin role
            if current_user.role != 'Admin':
                return jsonify({
                    "message": "Admin access required",
                    "error": "Forbidden"
                }), 403

            data = request.get_json()

            # Validate request data
            if not data or "id" not in data:
                return jsonify({
                    "message": "Missing entry ID",
                    "error": "Bad Request"
                }), 400

            try:
                # Find and delete entry
                entry = LeaderboardEntry.query.get(data['id'])
                if not entry:
                    return jsonify({
                        "message": "Entry not found", 
                        "error": "Not Found"
                    }), 404

                entry.delete()
                return jsonify({
                    "message": "Entry deleted successfully"
                }), 200

            except Exception as e:
                return jsonify({
                    "message": "Failed to delete entry",
                    "error": str(e)
                }), 500

        @token_required()
        def put(self):
            """Update a leaderboard entry (Admin only)"""
            current_user = g.current_user
            
            # Verify user has admin role
            if current_user.role != 'Admin':
                return jsonify({
                    "message": "Admin access required",
                    "error": "Forbidden"
                }), 403

            data = request.get_json()

            # Validate request data
            if not data or "id" not in data:
                return jsonify({
                    "message": "Missing entry ID",
                    "error": "Bad Request"
                }), 400

            try:
                # Find entry to update
                entry = LeaderboardEntry.query.get(data['id'])
                if not entry:
                    return jsonify({
                        "message": "Entry not found",
                        "error": "Not Found"
                    }), 404

                # Update score if provided and valid
                if 'score' in data:
                    score = int(data['score'])
                    if 0 <= score <= 100:
                        entry.score = score
                    else:
                        return jsonify({
                            "message": "Score must be between 0 and 100",
                            "error": "Bad Request"
                        }), 400

                # Update drawing name if provided
                if 'drawing_name' in data:
                    entry.drawing_name = data['drawing_name']

                entry.update()
                return jsonify(entry.read()), 200

            except Exception as e:
                return jsonify({
                    "message": "Failed to update entry",
                    "error": str(e)
                }), 500

    # Register CRUD resource with API
    api.add_resource(_CRUD, '/leaderboard')

## 2. Algorithmic code request
The API class and methods are defined to handle different types of HTTP requests (GET, POST, PUT, DELETE). Each method in the class corresponds to a specific CRUD operation.

#### Code showing this:

In [ ]:
@leaderboard_api.route('/api/leaderboard', methods=['GET'])
def get_leaderboard():
    try:
        entries = LeaderboardEntry.query.order_by( # query all entries in descending order of score (SQLAlchemy)
            LeaderboardEntry.score.desc()
        ).all()
        return jsonify([entry.read() for entry in entries]), 200 
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@leaderboard_api.route('/api/leaderboard', methods=['POST'])
def add_leaderboard_entry():
    try:
        data = request.get_json() # Get data from request
        if not data:
            return jsonify({"error": "No data provided"}), 400

        entry = LeaderboardEntry( # Create new entry
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name'),
            score=int(data.get('score', 0))
        )
        
        if entry.create():
            return jsonify({"message": "Entry added successfully"}), 201
        return jsonify({"error": "Failed to add entry"}), 400

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


@leaderboard_api.route('/api/leaderboard', methods=['PUT'])
def update_leaderboard_entry():
    try:
        data = request.get_json() # Get data from request
        if not data:
            return jsonify({"error": "No data provided"}), 400

        # Find existing entry
        existing_entry = LeaderboardEntry.query.filter_by(
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name')
        ).first()

        new_score = int(data.get('score', 0))

        # If entry exists, update it regardless of score
        if existing_entry:
            existing_entry.score = new_score  # Always update score
            db.session.commit()
            return jsonify({"message": "Score updated successfully"}), 200

        # If no existing entry, create new one
        entry = LeaderboardEntry(
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name'),
            score=new_score
        )
        
        if entry.create():
            return jsonify({"message": "New entry created"}), 201
        return jsonify({"error": "Failed to create entry"}), 400

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


@leaderboard_api.route('/api/leaderboard', methods=['DELETE'])
def delete_entry():
    try:
        # Try to get data from JSON body
        data = request.get_json()
        
        # Get profile_name and drawing_name from either JSON body or URL parameters
        profile_name = data.get('profile_name') if data else None
        drawing_name = data.get('drawing_name') if data else None

        # Validate input
        if not profile_name or not drawing_name:
            return jsonify({"error": "Missing profile_name or drawing_name"}), 400

        # Find and delete entry
        entry = LeaderboardEntry.query.filter_by(
            profile_name=profile_name,
            drawing_name=drawing_name
        ).first()

        if not entry:
            return jsonify({"error": "Entry not found"}), 404

        entry.delete()
        return jsonify({"message": "Entry deleted successfully"}), 200

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


## 3. Parameters and Return Type
The body of the request contains JSON data with the following fields:

- profile_name: The name of the player.
- drawing_name: The name of the drawing.
- score: The score achieved.

#### Example:

In [ ]:
# Example JSON payload 
{
    "drawing_name": "Sunset Beach",
    "score": 95
}

## 4. Call to Algorithm Request

## Definition of Code Block  
The `fetchLeaderboard` function sends a `GET` request to the API endpoint to fetch leaderboard data.  

## Call/Request to the Method  
The Fetch API is used to make the request, and the response is handled asynchronously.  

## Return/Response from the Method  
The response is parsed as JSON and passed to the `displayLeaderboard` function. Errors are logged and displayed in the DOM.  

## Handling Normal and Error Conditions  
The `submitScore` and `deleteEntry` functions handle changing data and trigger different responses based on normal and error conditions.  



#### Code:

In [ ]:
// Function to fetch leaderboard data from API
async function fetchLeaderboard() {
    const messageElement = document.getElementById('message');
    const leaderboardBody = document.getElementById('leaderboard-entries');

    // Clear existing entries
    leaderboardBody.innerHTML = '';

    try {
        // Call/Request to Method: Send GET request to API endpoint
        const response = await fetch(`${pythonURI}/api/leaderboard`, {
            method: "GET"  // GET request doesn't need authentication
        });

        // Check for failed response
        if (!response.ok) throw new Error('Failed to load leaderboard');
        
        // Return/Response: Parse JSON data from response
        const data = await response.json();

        // Normal Condition: Process and display data
        data
            .sort((a, b) => b.score - a.score)  // Sort by score descending
            .forEach((entry, index) => {
                const row = document.createElement('tr');
                row.innerHTML = `
                    <td><span style="color: ${index < 3 ? '#dc2626' : '#666'}">#${index + 1}</span></td>
                    <td>${entry.profile_name}</td>
                    <td>${entry.drawing_name}</td>
                    <td>${entry.score}</td>
                    <td>${entry.created_by ? 
                        `<button class="delete-btn" onclick="deleteEntry(${entry.id})">Delete</button>` 
                        : ''}</td>
                `;
                leaderboardBody.appendChild(row);
            });
        messageElement.textContent = '';

    } catch (error) {
        // Error Condition: Handle and display error
        console.error('Error:', error);
        messageElement.textContent = 'Error loading leaderboard';
        messageElement.className = 'message error';
    }
}

// Function to submit a new score
async function submitScore(event) {
    event.preventDefault();
    const messageElement = document.getElementById('message');
    
    // Get form data
    const drawingName = document.getElementById('drawingName').value.trim();
    const score = parseInt(document.getElementById('score').value);

    // Validate score
    if (score < 0 || score > 100) {
        messageElement.textContent = 'Score must be between 0 and 100';
        messageElement.className = 'message error';
        return;
    }

    try {
        // Call/Request to Method: Send POST request to API endpoint
        const response = await fetch(`${pythonURI}/api/leaderboard`, {
            credentials: 'include',
            headers: {
                'Content-Type': 'application/json'
            },
            method: "POST",
            body: JSON.stringify({
                drawing_name: drawingName,
                score: score
            })
        });

        // Return/Response: Parse JSON data from response
        const data = await response.json();
        
        //